- Update: 2023.04.11

# 0. 配置参数

- `rockauto`：ROCKAUTO.COM
- `manufacturer`：Manufacturer
- `part_type`：Part Type
- `serial`：可能是Rockauto的内部编号？

In [1]:
rockauto = 'https://www.rockauto.com/en/parts/'

manufacturer = 'dorman'
part_type = 'parking+brake+cable'
serial = '1696'

chromedriver = '../../chromedriver.exe'

In [2]:
from selenium.webdriver import ChromeOptions, Chrome
from selenium.webdriver.chrome.service import Service
import time

In [3]:
chrome_options = ChromeOptions()
chrome_options.add_experimental_option('excludeSwitches', ['enable-automation']) # Chrome正受到自动测试软件的控制
chrome_options.add_argument('--disable-blink-features=AutomationControlled') # window.navigator.webdriver=false
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36')
# chrome_options.add_argument('--proxy-server=http://a002.kdltps.com:15818')
# chrome_options.add_argument('blink-settings=imagesEnabled=false')

driver = Chrome(service=Service(chromedriver), options=chrome_options)
time.sleep(0.3)
driver.get(rockauto + manufacturer + ',' + part_type + ',' + serial)

In [4]:
from selenium.webdriver.common.by import By
import pandas as pd

In [5]:
list_part_number = [i.text for i in driver.find_elements(By.XPATH, '//*[@class="ranavnode"]')[1:]]
pd.DataFrame({'Part Number': list_part_number}).to_excel('./menu-' + manufacturer + '_' + part_type + '_' + serial + '.xlsx', index=False)
print('总数量：' + str(len(list_part_number)))
print()

总数量：2178



# 1. 爬取

In [ ]:
list_part_number = pd.read_excel('./menu-' + manufacturer + '_' + part_type + '_' + serial + '.xlsx')['Part Number'].to_list()

list_part_number

In [ ]:
df = pd.DataFrame(columns=['Part Number',
                           '标题',
                           '图片',
                           'OEM Number',
                           'Alternate Inventory Number',
                           '车型',
                           '链接',
                           '图片链接',
                           '配件-序号',
                           '配件-标题',
                           '配件-图片',
                           '配件-链接',
                           '配件-品类',
                           '配件-数量',
                           '配件-OEM Number',
                           '配件-Alternate Inventory Number',
                           '配件-图片链接'])

In [ ]:
i = 0
url = rockauto + manufacturer + ',' + list_part_number[i] + ',' + part_type + ',' + serial
driver.get(url)
time.sleep(0.3)

In [ ]:
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
WebDriverWait(driver, 60, 0.5, None).until(EC.presence_of_element_located((By.XPATH, '//*[@title="Buyer\'s Guide"]')))
buyers_guide = driver.find_element(By.XPATH, '//*[@title="Buyer\'s Guide"]')
buyers_guide.click()

WebDriverWait(driver, 60, 0.5, None).until(EC.presence_of_element_located((By.XPATH, '//*[@class="buyersguide-nested"]')))
buyers_guide = driver.find_element(By.XPATH, '//*[@class="buyersguide-nested"]').text.replace('\nPlease refer to catalog for application details.', '')

print(buyers_guide)

In [ ]:
info = driver.find_element(By.XPATH, '//*[@class="ra-btn ra-btn-moreinfo"]').get_attribute('href')
driver.get(info)

In [ ]:
WebDriverWait(driver, 60, 0.5, None).until(EC.presence_of_element_located((By.XPATH, '//*[@class="listing-text-row-moreinfo-truck"]')))
title = driver.find_element(By.XPATH, '//*[@class="listing-text-row-moreinfo-truck"]').text

title

In [ ]:
dict_alternate = {'alternate_inventory_number': '', 'oem_number': ''}
WebDriverWait(driver, 60, 0.5, None).until(EC.presence_of_element_located((By.XPATH, '//*[@aria-label="Alternate/OEM Part Number(s)"]')))
list_alternate = [j.text for j in driver.find_elements(By.XPATH, '//*[@aria-label="Alternate/OEM Part Number(s)"]')]
for j in range(len(list_alternate)):
    if '{Alternate Inventory Numbers: ' in list_alternate[j]:
        dict_alternate['alternate_inventory_number'] = list_alternate[j].replace('{Alternate Inventory Numbers: ', '').replace('}', '').replace(', ', ';')
    elif 'OEM / Interchange Numbers: ' in list_alternate[j]:
        dict_alternate['oem_number'] = list_alternate[j].replace('OEM / Interchange Numbers: ', '').replace(', ', ';')

dict_alternate

In [ ]:
WebDriverWait(driver, 60, 0.5, None).until(EC.presence_of_element_located((By.XPATH, '//*[@class="listing-inline-image listing-inline-image-moreinfo"]')))
pic = driver.find_element(By.XPATH, '//*[@class="listing-inline-image listing-inline-image-moreinfo"]').get_attribute('src')

pic

In [ ]:
df_temp = pd.DataFrame({'Part Number': [list_part_number[i]],
                        '标题': [title],
                        '图片': [''],
                        'OEM Number': [dict_alternate['oem_number']],
                        'Alternate Inventory Number': [dict_alternate['alternate_inventory_number']],
                        '车型': [buyers_guide],
                        '链接': [url],
                        '图片链接': [pic],
                        '配件-序号': [''],
                        '配件-标题': [''],
                        '配件-图片': [''],
                        '配件-链接': [''],
                        '配件-品类': [''],
                        '配件-数量': [''],
                        '配件-OEM Number': [''],
                        '配件-Alternate Inventory Number': [''],
                        '配件-图片链接': ['']})

df_temp

In [ ]:
list_component = driver.find_elements(By.XPATH, '//*[@aria-label="Components that come in this kit"]/table/tbody/tr/td/table/tbody')[1:]
if len(list_component) > 0:
    list_dict_component = []
    for j in range(len(list_component)):
        list_dict_component.append({'no': str(j+1),
                                    'title': list_component[j].find_element(By.XPATH, './tr/td[3]').text.replace(' Info', ''),
                                    'url': list_component[j].find_element(By.XPATH, './tr/td[3]/div[1]/a').get_attribute('href'),
                                    'category': list_component[j].find_element(By.XPATH, './tr/td[2]').text,
                                    'quantity': list_component[j].find_element(By.XPATH, './tr/td[1]').text,
                                    'oem_number': '',
                                    'alternate_inventory_number': '',
                                    'pic': ''})
    
    # = = = = = = = = = = = = = = = = = =
    
    for j in range(len(list_dict_component)):
        driver.get(list_dict_component[j]['url'])
        WebDriverWait(driver, 60, 0.5, None).until(EC.presence_of_element_located((By.XPATH, '//*[@aria-label="Alternate/OEM Part Number(s)"]')))
        list_alternate = [k.text for k in driver.find_elements(By.XPATH, '//*[@aria-label="Alternate/OEM Part Number(s)"]')]
        for k in range(len(list_alternate)):
            if '{Alternate Inventory Numbers: ' in list_alternate[k]:
                list_dict_component[j]['alternate_inventory_number'] = list_alternate[k].replace('{Alternate Inventory Numbers: ', '').replace('}', '').replace(', ', ';')
            elif 'OEM / Interchange Numbers: ' in list_alternate[k]:
                list_dict_component[j]['oem_number'] = list_alternate[k].replace('OEM / Interchange Numbers: ', '').replace(', ', ';')

        WebDriverWait(driver, 60, 0.5, None).until(EC.presence_of_element_located((By.XPATH, '//*[@class="listing-inline-image listing-inline-image-moreinfo"]')))
        list_dict_component[j]['pic'] = driver.find_element(By.XPATH, '//*[@class="listing-inline-image listing-inline-image-moreinfo"]').get_attribute('src')

    # = = = = = = = = = = = = = = = = = =
    
    for j in range(len(list_dict_component)):
        df_temp.loc[j, '配件-序号'] = list_dict_component[j]['no']
        df_temp.loc[j, '配件-标题'] = list_dict_component[j]['title']
        df_temp.loc[j, '配件-链接'] = list_dict_component[j]['url']
        df_temp.loc[j, '配件-品类'] = list_dict_component[j]['category']
        df_temp.loc[j, '配件-数量'] = list_dict_component[j]['quantity']
        df_temp.loc[j, '配件-OEM Number'] = list_dict_component[j]['oem_number']
        df_temp.loc[j, '配件-Alternate Inventory Number'] = list_dict_component[j]['alternate_inventory_number']
        df_temp.loc[j, '配件-图片链接'] = list_dict_component[j]['pic']
        
df_temp

In [ ]:
df = pd.concat([df, df_temp], ignore_index=True).fillna('')

driver.close()

df